In [3]:
# 완성버전

import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep

# 설정
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
current_time = datetime.now().strftime('%y-%m-%d')
search_url = "https://map.naver.com/v5/search"

def is_element_present(by, value):
    """주어진 요소가 페이지에 존재하는지 확인하는 함수"""
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:  # 예외 처리 시 사용
        return False

def wait_for_element(css_selector, timeout=60):
    """지정된 시간동안 요소를 기다림"""
    try:
        return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    except Exception as e:
        print(f"요소 '{css_selector}'를 찾지 못했습니다.")
        driver.quit()

def switch_frame(frame_name, timeout=60):
    """프레임 변경 (프레임이 로드될 때까지 대기)"""
    driver.switch_to.default_content()
    
    try:
        # 프레임이 존재하는지 확인하고, 해당 프레임이 로드될 때까지 대기
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"iframe#{frame_name}")))
        
        # 프레임이 준비되면 전환
        WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.ID, frame_name)))
        print(f"프레임 '{frame_name}'으로 성공적으로 전환되었습니다.")
        
    except Exception as e:
        print(f"프레임 '{frame_name}'으로 전환 중 오류 발생: {e}")


def first_search(search_query):
    """검색어로 검색 후 리뷰 추출"""
    driver.get(search_url)
    search_box = wait_for_element('div.input_box > input.input_search')
    
    # '검색어'로 검색
    search_box.send_keys(search_query + Keys.ENTER)
    sleep(3)
    
def clear_search():
    """검색창 비우기"""
    search_box = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search_box.send_keys(Keys.CONTROL + 'a')
    search_box.send_keys(Keys.BACKSPACE)

def fetch_reviews(search_query2):
    """검색어로 검색 후 리뷰 추출"""
    try:
        # 검색어 입력 및 검색 실행
        wait_for_element('div.input_box > input.input_search').send_keys(search_query2 + Keys.ENTER)
        sleep(3)
        
        # 'entryIframe' 존재 여부 확인
        if is_element_present(By.CSS_SELECTOR, 'iframe#entryIframe'):
            sleep(3)
            # 'entryIframe'이 존재하는 경우
            switch_frame('entryIframe')
            sleep(3)
            
            # 검색 결과 바로 가게 정보가 뜨는 경우
            print(f"'{search_query2}'에 대한 검색 결과가 바로 표시됨.")
            sleep(3)  # 결과 페이지 로딩 대기
            click_review_tab()
        
        elif is_element_present(By.CSS_SELECTOR, 'iframe#searchIframe'):
            sleep(3)
            # 'entryIframe'이 존재하지 않는 경우
            switch_frame('searchIframe')
            sleep(3)
            
            # 가게 검색 결과를 찾기
            results = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT, .place_bluelink.C6RjW")
            if results:
                for result in results:
                    if search_query2 in result.text:
                        print(f"'{search_query2}'에 해당하는 가게를 클릭합니다.")
                        result.click()
                        break
                else:
                    print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                    return []

                sleep(3)  # 결과 페이지 로딩 대기
                switch_frame('entryIframe')
                click_review_tab()
            
            else:
                print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                return []

        # "더보기" 클릭 루프
        for _ in range(20):
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, 'span.TeItc')
                driver.execute_script("arguments[0].click();", more_button)
                sleep(3)
            except:
                break
                
        # 리뷰 추출
        print("리뷰 추출을 시작합니다.")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = [span.text for span in soup.find_all("a", class_="pui__xtsQN-")]
        print(f"'{search_query2}'에 대한 리뷰 수집이 완료되었습니다.")
        return reviews

    except Exception as e:
        print(f"키워드 '{search_query2}' 처리 중 오류 발생: {e}")
        return []

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        review_tab = driver.find_element(By.XPATH, "//span[text()='리뷰']")
        review_tab.click()
        sleep(3)
    except Exception as e:
        print(f"리뷰 탭을 클릭하는 중 오류 발생: {e}")

def get_reviews_from_excel(detail_file_path):
    """엑셀 파일에서 키워드를 불러와 리뷰 수집"""
    # 세부 정보 파일에서 데이터 불러오기
    df_detail = pd.read_excel(detail_file_path)
    
    # 리뷰 데이터프레임 초기화
    reviews_list = []

    # 세부 정보의 검색어 및 검색결과에 대해 리뷰 수집
    for search_query, search_query2 in zip(df_detail['검색어'].dropna(), df_detail['검색결과'].dropna()):  # '검색어'와 '검색결과' 열을 기준으로 조회
        print(f"'{search_query}'로 검색 후 '{search_query2}'에 대한 리뷰 수집 중...")
        first_search(search_query)
        clear_search()
        reviews = fetch_reviews(search_query2)
        
        if reviews:  # 리뷰가 있는 경우에만 추가
            reviews_list.extend([{'검색어': search_query, '검색결과': search_query2, 'review': review} for review in reviews])

    return pd.DataFrame(reviews_list)

# 메인 실행 부분
detail_file = "../data/음식점_세부정보_24-08-28.xlsx"
df_reviews = get_reviews_from_excel(detail_file)
output_file = f'음식점_네이버_리뷰_{current_time}_2.xlsx'
df_reviews.to_excel(output_file, index=False)
print(f"리뷰 정보가 '{output_file}' 파일에 저장되었습니다.")

# 드라이버 종료 (필요한 경우)
driver.quit()

'성수카페거리'로 검색 후 '37.5 시그니처 성수'에 대한 리뷰 수집 중...
프레임 'searchIframe'으로 성공적으로 전환되었습니다.
'37.5 시그니처 성수'에 해당하는 가게를 클릭합니다.
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
리뷰 추출을 시작합니다.
'37.5 시그니처 성수'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '탐광'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'탐광'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'탐광'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '타이거풀 성수'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'타이거풀 성수'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'타이거풀 성수'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '성수다락'에 대한 리뷰 수집 중...
프레임 'searchIframe'으로 성공적으로 전환되었습니다.
'성수다락'에 해당하는 가게를 클릭합니다.
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
리뷰 추출을 시작합니다.
'성수다락'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 'bd버거 성수'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'bd버거 성수'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'bd버거 성수'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '네모네'에 대한 리뷰 수집 중...
프레임 'searchIframe'으로 성공적으로 전환되었습니다.
'네모네'에 해당하는 가게를 클릭합니다.
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
리뷰 추출을 시작합니다.
'네모네'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '모르므로'에 대한 리뷰 수집

In [6]:
# 추가 수정

import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep

# 설정
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
current_time = datetime.now().strftime('%y-%m-%d')
search_url = "https://map.naver.com/v5/search"

def is_element_present(by, value):
    """주어진 요소가 페이지에 존재하는지 확인하는 함수"""
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:  # 예외 처리 시 사용
        return False

def wait_for_element(css_selector, timeout=60):
    """지정된 시간동안 요소를 기다림"""
    try:
        return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    except Exception as e:
        print(f"요소 '{css_selector}'를 찾지 못했습니다.")
        driver.quit()

def switch_frame(frame_name, timeout=60):
    """프레임 변경 (프레임이 로드될 때까지 대기)"""
    driver.switch_to.default_content()
    
    try:
        # 프레임이 존재하는지 확인하고, 해당 프레임이 로드될 때까지 대기
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"iframe#{frame_name}")))
        
        # 프레임이 준비되면 전환
        WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.ID, frame_name)))
        print(f"프레임 '{frame_name}'으로 성공적으로 전환되었습니다.")
        
    except Exception as e:
        print(f"프레임 '{frame_name}'으로 전환 중 오류 발생: {e}")


def first_search(search_query):
    """검색어로 검색 후 리뷰 추출"""
    driver.get(search_url)
    search_box = wait_for_element('div.input_box > input.input_search')
    
    # '검색어'로 검색
    search_box.send_keys(search_query + Keys.ENTER)
    sleep(3)
    
def clear_search():
    """검색창 비우기"""
    search_box = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search_box.send_keys(Keys.CONTROL + 'a')
    search_box.send_keys(Keys.BACKSPACE)

def fetch_reviews(search_query2):
    """검색어로 검색 후 리뷰 추출"""
    try:
        # 검색어 입력 및 검색 실행
        wait_for_element('div.input_box > input.input_search').send_keys(search_query2 + Keys.ENTER)
        sleep(3)
        
        # 'entryIframe' 존재 여부 확인
        if is_element_present(By.CSS_SELECTOR, 'iframe#entryIframe'):
            # 'entryIframe'이 존재하는 경우
            switch_frame('entryIframe')
            
            # 검색 결과 바로 가게 정보가 뜨는 경우
            print(f"'{search_query2}'에 대한 검색 결과가 바로 표시됨.")
            sleep(3)  # 결과 페이지 로딩 대기
            click_review_tab()
        
        elif is_element_present(By.CSS_SELECTOR, 'iframe#searchIframe'):
            # 'entryIframe'이 존재하지 않는 경우
            switch_frame('searchIframe')
            
            # 가게 검색 결과를 찾기
            results = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT, .place_bluelink.C6RjW")
            if results:
                for result in results:
                    if search_query2 in result.text:
                        print(f"'{search_query2}'에 해당하는 가게를 클릭합니다.")
                        result.click()
                        break
                else:
                    print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                    return []

                sleep(3)  # 결과 페이지 로딩 대기
                switch_frame('entryIframe')
                click_review_tab()
            
            else:
                print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                return []

        # "더보기" 클릭 루프 (최대 1번)
        for _ in range(20):
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, 'span.TeItc')
                driver.execute_script("arguments[0].click();", more_button)
                sleep(3)
            except:
                break
                
        # 리뷰 추출
        print("리뷰 추출을 시작합니다.")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = [span.text for span in soup.find_all("a", class_="pui__xtsQN-")]
        print(f"'{search_query2}'에 대한 리뷰 수집이 완료되었습니다.")
        return reviews

    except Exception as e:
        print(f"키워드 '{search_query2}' 처리 중 오류 발생: {e}")
        return []

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        sleep(3)
        review_tab = driver.find_element(By.XPATH, "//span[text()='리뷰']")
        review_tab.click()
        sleep(3)
    except Exception as e:
        print(f"리뷰 탭을 클릭하는 중 오류 발생: {e}")

def get_reviews_from_excel(detail_file_path):
    """엑셀 파일에서 키워드를 불러와 리뷰 수집"""
    # 세부 정보 파일에서 데이터 불러오기
    df_detail = pd.read_excel(detail_file_path)
    
    # 리뷰 데이터프레임 초기화
    reviews_list = []

    # 세부 정보의 검색어 및 검색결과에 대해 리뷰 수집
    for search_query, search_query2 in zip(df_detail['검색어'].dropna(), df_detail['검색결과'].dropna()):  # '검색어'와 '검색결과' 열을 기준으로 조회
        print(f"'{search_query}'로 검색 후 '{search_query2}'에 대한 리뷰 수집 중...")
        first_search(search_query)
        clear_search()
        reviews = fetch_reviews(search_query2)
        
        if reviews:  # 리뷰가 있는 경우에만 추가
            reviews_list.extend([{'검색어': search_query, '검색결과': search_query2, 'review': review} for review in reviews])

    return pd.DataFrame(reviews_list)

# 메인 실행 부분
detail_file = "../data/음식점_세부정보_24-08-31.xlsx"
df_reviews = get_reviews_from_excel(detail_file)
output_file = f'../data/음식점_네이버_리뷰_{current_time}.xlsx'
df_reviews.to_excel(output_file, index=False)
print(f"리뷰 정보가 '{output_file}' 파일에 저장되었습니다.")

# 드라이버 종료 (필요한 경우)
driver.quit()

'성수카페거리'로 검색 후 '마야'에 대한 리뷰 수집 중...
프레임 'searchIframe'으로 성공적으로 전환되었습니다.
'마야'에 해당하는 가게를 클릭합니다.
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
리뷰 추출을 시작합니다.
'마야'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '봄의정원 성수점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'봄의정원 성수점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'봄의정원 성수점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '사위식당 성수점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'사위식당 성수점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'사위식당 성수점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 'LOOOP 루프 베이커리 카페'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'LOOOP 루프 베이커리 카페'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'LOOOP 루프 베이커리 카페'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '뚝도농원 성수본점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'뚝도농원 성수본점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'뚝도농원 성수본점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 'bd버거 성수'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'bd버거 성수'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'bd버거 성수'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '탐광'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로

IllegalCharacterError: 양을 맘껏 먹을 수 있어  학생들이 좋아해요 cannot be used in worksheets.

In [ ]:
# 추가 수정

import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep

# 설정
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
current_time = datetime.now().strftime('%y-%m-%d')
search_url = "https://map.naver.com/v5/search"

def is_element_present(by, value):
    """주어진 요소가 페이지에 존재하는지 확인하는 함수"""
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:  # 예외 처리 시 사용
        return False

def wait_for_element(css_selector, timeout=60):
    """지정된 시간동안 요소를 기다림"""
    try:
        return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    except Exception as e:
        print(f"요소 '{css_selector}'를 찾지 못했습니다.")
        driver.quit()

def switch_frame(frame_name, timeout=60):
    """프레임 변경 (프레임이 로드될 때까지 대기)"""
    driver.switch_to.default_content()
    
    try:
        # 프레임이 존재하는지 확인하고, 해당 프레임이 로드될 때까지 대기
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"iframe#{frame_name}")))
        
        # 프레임이 준비되면 전환
        WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.ID, frame_name)))
        print(f"프레임 '{frame_name}'으로 성공적으로 전환되었습니다.")
        
    except Exception as e:
        print(f"프레임 '{frame_name}'으로 전환 중 오류 발생: {e}")


def first_search(search_query):
    """검색어로 검색 후 리뷰 추출"""
    driver.get(search_url)
    search_box = wait_for_element('div.input_box > input.input_search')
    
    # '검색어'로 검색
    search_box.send_keys(search_query + Keys.ENTER)
    sleep(3)
    
def clear_search():
    """검색창 비우기"""
    search_box = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search_box.send_keys(Keys.CONTROL + 'a')
    search_box.send_keys(Keys.BACKSPACE)

def fetch_reviews(search_query2):
    """검색어로 검색 후 리뷰 추출"""
    try:
        # 검색어 입력 및 검색 실행
        wait_for_element('div.input_box > input.input_search').send_keys(search_query2 + Keys.ENTER)
        sleep(3)
        
        # 'entryIframe' 존재 여부 확인
        if is_element_present(By.CSS_SELECTOR, 'iframe#entryIframe'):
            # 'entryIframe'이 존재하는 경우
            switch_frame('entryIframe')
            
            # 검색 결과 바로 가게 정보가 뜨는 경우
            print(f"'{search_query2}'에 대한 검색 결과가 바로 표시됨.")
            sleep(3)  # 결과 페이지 로딩 대기
            click_review_tab()
        
        elif is_element_present(By.CSS_SELECTOR, 'iframe#searchIframe'):
            # 'entryIframe'이 존재하지 않는 경우
            switch_frame('searchIframe')
            
            # 가게 검색 결과를 찾기
            results = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT, .place_bluelink.C6RjW")
            if results:
                for result in results:
                    if search_query2 in result.text:
                        print(f"'{search_query2}'에 해당하는 가게를 클릭합니다.")
                        result.click()
                        break
                else:
                    print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                    return []

                sleep(3)  # 결과 페이지 로딩 대기
                switch_frame('entryIframe')
                click_review_tab()
            
            else:
                print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                return []

        # "더보기" 클릭 루프 (최대 1번)
        for _ in range(10):
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, 'span.TeItc')
                driver.execute_script("arguments[0].click();", more_button)
                sleep(3)
            except:
                break
                
        # 리뷰 추출
        print("리뷰 추출을 시작합니다.")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = [span.text for span in soup.find_all("a", class_="pui__xtsQN-")]
        print(f"'{search_query2}'에 대한 리뷰 수집이 완료되었습니다.")
        return reviews

    except Exception as e:
        print(f"키워드 '{search_query2}' 처리 중 오류 발생: {e}")
        return []

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        sleep(3)
        review_tab = driver.find_element(By.XPATH, "//span[text()='리뷰']")
        review_tab.click()
        sleep(3)
    except Exception as e:
        print(f"리뷰 탭을 클릭하는 중 오류 발생: {e}")

def get_reviews_from_excel(detail_file_path):
    """엑셀 파일에서 키워드를 불러와 리뷰 수집"""
    # 세부 정보 파일에서 데이터 불러오기
    df_detail = pd.read_excel(detail_file_path)
    
    # 리뷰 데이터프레임 초기화
    reviews_list = []

    # 세부 정보의 검색어 및 검색결과에 대해 리뷰 수집
    for search_query, search_query2 in zip(df_detail['검색어'].dropna(), df_detail['검색결과'].dropna()):  # '검색어'와 '검색결과' 열을 기준으로 조회
        print(f"'{search_query}'로 검색 후 '{search_query2}'에 대한 리뷰 수집 중...")
        first_search(search_query)
        clear_search()
        reviews = fetch_reviews(search_query2)
        
        if reviews:  # 리뷰가 있는 경우에만 추가
            reviews_list.extend([{'검색어': search_query, '검색결과': search_query2, 'review': review} for review in reviews])

    return pd.DataFrame(reviews_list)

# 메인 실행 부분
detail_file = "../data/음식점_세부정보_24-08-31.xlsx"
df_reviews = get_reviews_from_excel(detail_file)
output_file = f'../data/음식점_네이버_리뷰_{current_time}.xlsx'
df_reviews.to_excel(output_file, index=False)
print(f"리뷰 정보가 '{output_file}' 파일에 저장되었습니다.")

# 드라이버 종료 (필요한 경우)
driver.quit()

In [3]:
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from time import sleep

# 설정
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
current_time = datetime.now().strftime('%y-%m-%d')
search_url = "https://map.naver.com/v5/search"

def is_element_present(by, value):
    """주어진 요소가 페이지에 존재하는지 확인하는 함수"""
    try:
        driver.find_element(by, value)
        return True
    except NoSuchElementException:  # 예외 처리 시 사용
        return False

def wait_for_element(css_selector, timeout=60):
    """지정된 시간동안 요소를 기다림"""
    try:
        return WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    except Exception as e:
        print(f"요소 '{css_selector}'를 찾지 못했습니다.")
        driver.quit()

def switch_frame(frame_name, timeout=60):
    """프레임 변경 (프레임이 로드될 때까지 대기)"""
    driver.switch_to.default_content()
    
    try:
        # 프레임이 존재하는지 확인하고, 해당 프레임이 로드될 때까지 대기
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"iframe#{frame_name}")))
        
        # 프레임이 준비되면 전환
        WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.ID, frame_name)))
        print(f"프레임 '{frame_name}'으로 성공적으로 전환되었습니다.")
        
    except Exception as e:
        print(f"프레임 '{frame_name}'으로 전환 중 오류 발생: {e}")


def first_search(search_query):
    """검색어로 검색 후 리뷰 추출"""
    driver.get(search_url)
    search_box = wait_for_element('div.input_box > input.input_search')
    
    # '검색어'로 검색
    search_box.send_keys(search_query + Keys.ENTER)
    sleep(3)  # 검색 후 페이지가 로딩되도록 대기 시간 증가
    
def clear_search():
    """검색창 비우기"""
    search_box = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search_box.send_keys(Keys.CONTROL + 'a')
    search_box.send_keys(Keys.BACKSPACE)
    sleep(3)  # 검색창 초기화 후 대기

def fetch_reviews(search_query2):
    """검색어로 검색 후 리뷰 추출"""
    try:
        # 검색어 입력 및 검색 실행
        wait_for_element('div.input_box > input.input_search').send_keys(search_query2 + Keys.ENTER)
        sleep(3)  # 검색 후 페이지가 로딩되도록 대기 시간 증가
        
        # 'entryIframe' 존재 여부 확인
        if is_element_present(By.CSS_SELECTOR, 'iframe#entryIframe'):
            # 'entryIframe'이 존재하는 경우
            switch_frame('entryIframe')
            
            # 검색 결과 바로 가게 정보가 뜨는 경우
            print(f"'{search_query2}'에 대한 검색 결과가 바로 표시됨.")
            sleep(3)  # 결과 페이지 로딩 대기 시간 증가
            click_review_tab()
        
        elif is_element_present(By.CSS_SELECTOR, 'iframe#searchIframe'):
            # 'entryIframe'이 존재하지 않는 경우
            switch_frame('searchIframe')
            sleep(3)
            
            # 가게 검색 결과를 찾기
            results = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT, .place_bluelink.C6RjW")
            if results:
                for result in results:
                    if search_query2 in result.text:
                        print(f"'{search_query2}'에 해당하는 가게를 클릭합니다.")
                        result.click()
                        break
                else:
                    print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                    return []

                sleep(5)  # 결과 페이지 로딩 대기 시간 증가
                switch_frame('entryIframe')
                click_review_tab()
            
            else:
                print(f"'{search_query2}'에 대한 검색 결과를 목록에서 찾을 수 없습니다.")
                return []

        # "더보기" 클릭 루프 (최대 1번)
        for _ in range(2):
            try:
                more_button = driver.find_element(By.CSS_SELECTOR, 'span.TeItc')
                driver.execute_script("arguments[0].click();", more_button)
                sleep(2)  # "더보기" 클릭 후 대기 시간 증가
            except:
                break
                
        # 리뷰 추출
        print("리뷰 추출을 시작합니다.")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        reviews = [span.text for span in soup.find_all("a", class_="pui__xtsQN-")]
        print(f"'{search_query2}'에 대한 리뷰 수집이 완료되었습니다.")
        return reviews

    except Exception as e:
        print(f"키워드 '{search_query2}' 처리 중 오류 발생: {e}")
        return []

def click_review_tab():
    """리뷰 탭 클릭"""
    try:
        sleep(3)  # 탭 클릭 전에 대기 시간 증가
        review_tab = driver.find_element(By.XPATH, "//span[text()='리뷰']")
        review_tab.click()
        sleep(3)  # 리뷰 탭 클릭 후 대기 시간 증가
    except Exception as e:
        print(f"리뷰 탭을 클릭하는 중 오류 발생: {e}")

def clean_review_text(review):
    """리뷰 텍스트에서 한글, 숫자, 영어만 남기고 나머지 제거"""
    return re.sub(r'[^가-힣0-9a-zA-Z\s]', '', review)

def get_reviews_from_excel(detail_file_path):
    """엑셀 파일에서 키워드를 불러와 리뷰 수집"""
    # 세부 정보 파일에서 데이터 불러오기
    df_detail = pd.read_excel(detail_file_path)
    
    # 리뷰 데이터프레임 초기화
    reviews_list = []

    # 세부 정보의 검색어 및 검색결과에 대해 리뷰 수집
    for search_query, search_query2 in zip(df_detail['검색어'].dropna(), df_detail['검색결과'].dropna()):  # '검색어'와 '검색결과' 열을 기준으로 조회
        print(f"'{search_query}'로 검색 후 '{search_query2}'에 대한 리뷰 수집 중...")
        first_search(search_query)
        clear_search()
        reviews = fetch_reviews(search_query2)
        
        if reviews:  # 리뷰가 있는 경우에만 추가
            cleaned_reviews = [clean_review_text(review) for review in reviews]  # 리뷰에서 한글, 숫자, 영어만 남기고 정리
            reviews_list.extend([{'검색어': search_query, '검색결과': search_query2, 'review': review} for review in cleaned_reviews])

    return pd.DataFrame(reviews_list)

# 메인 실행 부분
detail_file = "../data/음식점_세부정보_24-08-31.xlsx"
# detail_file = "../data/숙박시설_세부정보_24-08-31.xlsx"
# detail_file = "../data/카페_세부정보_24-08-31.xlsx"

df_reviews = get_reviews_from_excel(detail_file)
output_file = f'../data/음식점_네이버_리뷰_{current_time}_1.xlsx'
# output_file = f'../data/숙박시설_네이버_리뷰_{current_time}.xlsx'
# output_file = f'../data/카페_네이버_리뷰_{current_time}.xlsx'

df_reviews.to_excel(output_file, index=False)
print(f"리뷰 정보가 '{output_file}' 파일에 저장되었습니다.")

# 드라이버 종료 (필요한 경우)
driver.quit()

'성수카페거리'로 검색 후 '마야'에 대한 리뷰 수집 중...
프레임 'searchIframe'으로 성공적으로 전환되었습니다.
'마야'에 해당하는 가게를 클릭합니다.
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
리뷰 추출을 시작합니다.
'마야'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '봄의정원 성수점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'봄의정원 성수점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'봄의정원 성수점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '사위식당 성수점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'사위식당 성수점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'사위식당 성수점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 'LOOOP 루프 베이커리 카페'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'LOOOP 루프 베이커리 카페'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'LOOOP 루프 베이커리 카페'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '뚝도농원 성수본점'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'뚝도농원 성수본점'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'뚝도농원 성수본점'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 'bd버거 성수'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로 성공적으로 전환되었습니다.
'bd버거 성수'에 대한 검색 결과가 바로 표시됨.
리뷰 추출을 시작합니다.
'bd버거 성수'에 대한 리뷰 수집이 완료되었습니다.
'성수카페거리'로 검색 후 '탐광'에 대한 리뷰 수집 중...
프레임 'entryIframe'으로